In [1]:
import os
os.environ["OPENAI_API_KEY"] = "sk-xxx"

In [17]:
top_k = 3
dummy = "dummy"

In [21]:
import langchain
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

def encode_text(text):
    embeds = embeddings.embed_query(text)
    return embeds

/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
query = "Who is the lead actor in the movie Bahubali?"

In [23]:
query_embeds = encode_text(query)

In [24]:
from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
    "bolt://localhost:7687",
  auth=basic_auth("neo4j", "$$1234bala"))

In [41]:
graph_similarity = """// 1 - Find the movie nodes with similar embeddings
MATCH (movieEmb:Movie {title: $dummy_title})
WITH movieEmb
ORDER BY movieEmb.rank // Assuming rank or some other property to initiate the search
LIMIT 1
CALL db.index.vector.queryNodes('desc-embeddings',$k, movieEmb.embedding) YIELD node, score
WHERE node:Movie
// 2 - Retrieve all related data of the movie
OPTIONAL MATCH (node)-[:DIRECTED_BY]->(d:Director)
OPTIONAL MATCH (node)-[:FEATURES_ACTOR]->(a:Actor)
OPTIONAL MATCH (node)-[:BELONGS_TO_GENRE]->(g:Genre)
OPTIONAL MATCH (node)-[:RELEASED_IN]->(y:Year)
// 3 - Prepare results
RETURN node.title AS movie, 
       node.description AS desc,
       collect(DISTINCT d.name) AS directors, 
       collect(DISTINCT a.name) AS actors, 
       collect(DISTINCT g.name) AS genres, 
       y.year AS releaseYear, 
       score 
ORDER BY score DESC LIMIT $k;"""

In [35]:
def create_dummy_movie_node(tx, movie):
    query = (
        "CREATE (:Movie {"
        "    rank: toInteger($rank), "
        "    title: $title, "
        "    description: $description, "
        "    year: toInteger($year), "
        "    runtime: toInteger($runtime), "
        "    rating: toFloat($rating), "
        "    votes: toInteger($votes), "
        "    revenue: toFloat($revenue), "
        "    metascore: toInteger($metascore) "
        "})"
    )
    tx.run(query, rank=movie['rank'], title=movie['title'], description=movie['description'], year=movie['year'], runtime=movie['runtime'], rating=movie['rating'], votes=movie['votes'], revenue=movie['revenue'], metascore=movie['metascore'], embedding=movie['embedding'])

In [36]:
# match (m:Movie { title: value.Title })
def add_embed_movie_node(tx, movie):
    query = (
        "match (m:Movie { title: $title })"
        "set m.embedding = $embedding"
    )
    tx.run(query, rank=movie['rank'], title=movie['title'], description=movie['description'], year=movie['year'], runtime=movie['runtime'], rating=movie['rating'], votes=movie['votes'], revenue=movie['revenue'], metascore=movie['metascore'], embedding=movie['embedding'])

In [37]:
movie ={"rank":6, "title":"Dummy Node 007", "description":"NA", "year":1999, "runtime":120,"rating": 6.7, "votes":1000, "revenue":100, "metascore":100, "embedding":query_embeds}

In [43]:
with driver.session() as session:
    mov = session.write_transaction(create_dummy_movie_node, movie)
    # for record in mov:
    #     print(record["p"])

driver.close()

with driver.session() as session:
    mov2 = session.write_transaction(add_embed_movie_node, movie)

driver.close()

/var/folders/lc/p2v9_j0j0fxdgkt03r9sw75r0000gn/T/ipykernel_45437/3841302031.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
/var/folders/lc/p2v9_j0j0fxdgkt03r9sw75r0000gn/T/ipykernel_45437/3841302031.py:2: DeprecationWarning: write_transaction has been renamed to execute_write
  mov = session.write_transaction(create_dummy_movie_node, movie)
/var/folders/lc/p2v9_j0j0fxdgkt03r9sw75r0000gn/T/ipykernel_45437/3841302031.py:8: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
/var/folders/lc/p2v9_j0j0fxdgkt03r9sw75r0000gn/T/ipykernel_45437/3841302031.py:9: DeprecationWarning: write_transaction has been renamed to execute_write
  mov2 = session.write_transaction(add_embed_movie_node, movie)


In [46]:
with driver.session(database="neo4j") as session:
  results = session.read_transaction(
    lambda tx: tx.run(graph_similarity,dummy_title=movie["title"],k=top_k).data())

  # print(results)
  results.pop(0)
  context = "Related Movies:" + "\n".join(["title: " + record['movie'] + "\n" + "Description: " + record['desc'] + "\n" + "Director: " + "".join([d for d in record['directors']])+ "\n" + "Actors: " + "".join([d for d in record['actors']]) + "\n" + "Year: " + record['releaseYear'] for record in results ])

driver.close()

print(context)

Related Movies:title: Bahubali: The Beginning
Description: In ancient India, an adventurous and daring man becomes involved in a decades old feud between two warring people.
Director: S.S. Rajamouli
Actors: Anushka Shetty,Tamannaah BhatiaPrabhasRana Daggubati
Year: 2015


/var/folders/lc/p2v9_j0j0fxdgkt03r9sw75r0000gn/T/ipykernel_45437/1295354669.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session(database="neo4j") as session:
/var/folders/lc/p2v9_j0j0fxdgkt03r9sw75r0000gn/T/ipykernel_45437/1295354669.py:2: DeprecationWarning: read_transaction has been renamed to execute_read
  results = session.read_transaction(


In [48]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0, model="gpt-4")

template = """ 
    You are a movie research assistant tasked with providing detailed information about films, actors, directors, and genres based on a comprehensive movie database.
    Utilize the following context from the knowledge graph to answer the inquiries presented at the end.
    Strive to maintain the integrity of the context in your responses for accuracy. Do not alter the context unless absolutely necessary.
    If the answer is not available in the knowledge graph, admit the limitation rather than fabricating a response.
    ----
    {context}
    ----
    Each answer should conclude with metadata referencing the relevant movie information in the format (title, year, director, genres, actors).
    For instance, if the context includes metadata: (title:'Inception', year:2010, director:'Christopher Nolan', genres:'Sci-Fi, Thriller', actors:'Leonardo DiCaprio, Joseph Gordon-Levitt'), your response should display ('Inception', 2010, 'Christopher Nolan', 'Sci-Fi, Thriller', 'Leonardo DiCaprio, Joseph Gordon-Levitt').
    
    Question:```{question}```  """
prompt = PromptTemplate.from_template(template)

theprompt = prompt.format_prompt(question=query,context=context)

#print(theprompt)

llm(theprompt.to_messages())

AIMessage(content='The lead actor in the movie Bahubali: The Beginning is Prabhas. (Bahubali: The Beginning, 2015, S.S. Rajamouli, Anushka Shetty, Tamannaah Bhatia, Prabhas, Rana Daggubati)', additional_kwargs={}, example=False)

In [49]:
def del_dummy_movie_node(tx, movie):
    query = (
        "MATCH (m:Movie {title: $title}) "
        "DELETE m"
    )
    tx.run(query, title=movie['title'])

In [52]:
with driver.session() as session:
    mov = session.write_transaction(del_dummy_movie_node,movie)

driver.close()

/var/folders/lc/p2v9_j0j0fxdgkt03r9sw75r0000gn/T/ipykernel_45437/481839789.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
/var/folders/lc/p2v9_j0j0fxdgkt03r9sw75r0000gn/T/ipykernel_45437/481839789.py:2: DeprecationWarning: write_transaction has been renamed to execute_write
  mov = session.write_transaction(del_dummy_movie_node,movie)
